In [1]:
import numpy as np 
import pandas as pd 

import matplotlib.pyplot as plt
#import plotly.express as px
import seaborn as sns

from lightgbm import LGBMClassifier
from sklearn.ensemble import GradientBoostingClassifier

from sklearn.model_selection import StratifiedKFold
from collections import Counter
from imblearn.under_sampling import RandomUnderSampler
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix, \
                            accuracy_score, f1_score, average_precision_score, \
                            fbeta_score, precision_recall_curve, auc
from sklearn.metrics import matthews_corrcoef

from sklearn.model_selection import train_test_split

import optuna
from optuna.samplers import TPESampler
sns.set(style='white', context='notebook', palette='deep')
%config Completer.use_jedi = False

In [2]:
csl_df = pd.read_csv('../../data/csl/CSL_hety_PI_comp.csv', index_col=0)
X = csl_df.drop(['high_EBL','Sitenum'], axis=1, inplace=False)
y = csl_df['high_EBL'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=7)
X_train                                                    

,transfus_yes,Accrete,AdmBishop,Admcervpos,Admconsistency,Admcontract,AdmDBP,Admefface,Admpresent,AdmSBP,...,TD_nos,ThreatenedPB,threatpb9,UnspecHBP,uscar,version9,new_age,new_BMI,new_high_Age,new_high_BMI
MomID,,,,,,,,,,,,,,,,,,,,,
44-01170,0,0,8,8,8,2.5,76,50.0,77,139.0,...,0,0,0,0,0,0,25,24,1,1
41-42094,0,0,11,1,3,2.9,79,90.0,77,122.0,...,0,0,0,0,0,0,27,21,1,1
48-15713,0,0,8,8,8,66.0,90,100.0,77,127.0,...,0,0,0,0,0,0,18,26,0,1
49-04689,1,0,8,2,3,3.0,75,70.0,1,118.0,...,0,0,0,0,0,0,30,24,1,1
49-02986,0,0,8,1,3,1.0,81,100.0,1,128.0,...,0,0,0,0,0,0,26,24,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49-22617,0,0,8,8,8,1.0,88,0.0,77,125.0,...,0,0,0,0,1,0,31,33,1,1
45-01038,0,0,8,3,3,2.0,79,50.0,1,106.0,...,0,0,0,0,0,0,31,24,1,1
52-04194,0,0,8,8,8,66.0,74,70.0,1,124.4,...,0,0,0,0,0,0,30,21,1,1


<h1 align='center'>Exploratory Data Analysis</h1>

In [8]:
plt.figure(figsize=(20,10))
sns.barplot(csl_df['new_age'],csl_df['high_EBL'])
plt.show()

NameError: name 'plt' is not defined

In [7]:
sns.boxplot(csl_df['new_age'])
plt.show()

NameError: name 'sns' is not defined

In [6]:
plt.figure(figsize=(20,10))
sns.barplot(csl_df['new_BMI'],csl_df['high_EBL'])
plt.show()

NameError: name 'plt' is not defined

In [5]:
sns.boxplot(csl_df['new_BMI'])
plt.show()

NameError: name 'sns' is not defined

<h1 align='center'>Modelling</h1>

In [7]:
print(X_train.shape)
print(X.shape)
print(y.shape)

(95540, 195)
(136487, 195)
(136487,)


Here we define our validation set

<h1 align='center'>Modelling Using Undersampling</h1>

We will use `imblearn`'s RandomUnderSampler to undersample from the majority class so that they match

In [8]:
sampler = RandomUnderSampler(sampling_strategy=1.0, random_state=7)
X_rus, y_rus = sampler.fit_resample(X_train, y_train)

# Try with no undersampling
#X_rus, y_rus = X_train, y_train

In [9]:
print(X_rus.shape)
print(y_rus.shape)
print(np.bincount(y_rus))

(8064, 195)
(8064,)
[4032 4032]


<h1 align='center'>Basic Gradient Boosting</h1>

In [10]:
%%time
from sklearn.metrics import matthews_corrcoef
from sklearn.utils import class_weight
gb_rus = GradientBoostingClassifier()   # If not undersampling
gb_rus.fit(X_rus, y_rus)
print(gb_rus.get_params())
y_pred = gb_rus.predict(X_test)
probs = gb_rus.predict_proba(X_test)
prob1 = probs[:, 1]  # Only positives
precision, recall, pr_thresh = precision_recall_curve(y_test, prob1)
print(confusion_matrix(y_pred=y_pred, y_true=y_test))
print(classification_report(y_test, y_pred))
print(f'ROC AUC prob1: {roc_auc_score(y_test, prob1)}')
print(f'MCC: {matthews_corrcoef(y_test, y_pred)}')
print(f'F2: {fbeta_score(y_test, y_pred, beta=2.0, average=None)[1]}')
print(f'PR_AUC: {auc(recall, precision)}')

{'ccp_alpha': 0.0, 'criterion': 'friedman_mse', 'init': None, 'learning_rate': 0.1, 'loss': 'deviance', 'max_depth': 3, 'max_features': None, 'max_leaf_nodes': None, 'min_impurity_decrease': 0.0, 'min_impurity_split': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 100, 'n_iter_no_change': None, 'presort': 'deprecated', 'random_state': None, 'subsample': 1.0, 'tol': 0.0001, 'validation_fraction': 0.1, 'verbose': 0, 'warm_start': False}
[[30535  8684]
 [  169  1559]]
              precision    recall  f1-score   support

           0       0.99      0.78      0.87     39219
           1       0.15      0.90      0.26      1728

    accuracy                           0.78     40947
   macro avg       0.57      0.84      0.57     40947
weighted avg       0.96      0.78      0.85     40947

ROC AUC prob1: 0.8912720889841753
MCC: 0.31601897576278604
F2: 0.4543864762459925
PR_AUC: 0.2663190042652628
CPU times: user 3.82 s, sys: 91.5 ms, t

In [11]:
%%time
# Not undersampled
from sklearn.metrics import matthews_corrcoef
from sklearn.utils import class_weight
gb_weight = GradientBoostingClassifier()  
weights = class_weight.compute_sample_weight('balanced', y=y_train)
gb_weight.fit(X_train, y_train, sample_weight=weights)
print(gb_weight.get_params())
y_pred = gb_weight.predict(X_test)
probs = gb_weight.predict_proba(X_test)
prob1 = probs[:, 1]  # Only positives
precision, recall, pr_thresh = precision_recall_curve(y_test, prob1)
print(confusion_matrix(y_pred=y_pred, y_true=y_test))
print(classification_report(y_test, y_pred))
print(f'ROC AUC prob1: {roc_auc_score(y_test, prob1)}')
print(f'MCC: {matthews_corrcoef(y_test, y_pred)}')
print(f'F2: {fbeta_score(y_test, y_pred, beta=2.0, average=None)[1]}')
print(f'PR_AUC: {auc(recall, precision)}')

{'ccp_alpha': 0.0, 'criterion': 'friedman_mse', 'init': None, 'learning_rate': 0.1, 'loss': 'deviance', 'max_depth': 3, 'max_features': None, 'max_leaf_nodes': None, 'min_impurity_decrease': 0.0, 'min_impurity_split': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 100, 'n_iter_no_change': None, 'presort': 'deprecated', 'random_state': None, 'subsample': 1.0, 'tol': 0.0001, 'validation_fraction': 0.1, 'verbose': 0, 'warm_start': False}
[[30530  8689]
 [  169  1559]]
              precision    recall  f1-score   support

           0       0.99      0.78      0.87     39219
           1       0.15      0.90      0.26      1728

    accuracy                           0.78     40947
   macro avg       0.57      0.84      0.57     40947
weighted avg       0.96      0.78      0.85     40947

ROC AUC prob1: 0.8949700512459475
MCC: 0.3159084300584172
F2: 0.45425407925407923
PR_AUC: 0.28249431802839825
CPU times: user 42.8 s, sys: 159 ms, t

In [12]:
%%time
# Not undersampled - many parameters
from sklearn.metrics import matthews_corrcoef
from sklearn.utils import class_weight
gb_weight = GradientBoostingClassifier(learning_rate=0.2, max_features = 15, max_leaf_nodes=100)  
#'learning_rate': 0.1, 'loss': 'deviance', 'max_depth': 3, 'max_features': None, 'max_leaf_nodes': None, 'min_impurity_decrease': 0.0, 'min_impurity_split': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 100, 'n_iter_no_change': None, 'presort': 'deprecated', 'random_state': None, 'subsample': 1.0, 'tol': 0.0001, 'validation_fraction': 0.1, 'verbose': 0, 'warm_start': False
weights = class_weight.compute_sample_weight('balanced', y=y_train)
gb_weight.fit(X_train, y_train, sample_weight=weights)
print(gb_weight.get_params())
y_pred = gb_weight.predict(X_test)
probs = gb_weight.predict_proba(X_test)
prob1 = probs[:, 1]  # Only positives
precision, recall, pr_thresh = precision_recall_curve(y_test, prob1)
print(confusion_matrix(y_pred=y_pred, y_true=y_test))
print(classification_report(y_test, y_pred))
print(f'ROC AUC prob1: {roc_auc_score(y_test, prob1)}')
print(f'MCC: {matthews_corrcoef(y_test, y_pred)}')
print(f'F2: {fbeta_score(y_test, y_pred, beta=2.0, average=None)[1]}')
print(f'PR_AUC: {auc(recall, precision)}')

{'ccp_alpha': 0.0, 'criterion': 'friedman_mse', 'init': None, 'learning_rate': 0.2, 'loss': 'deviance', 'max_depth': 3, 'max_features': 15, 'max_leaf_nodes': 100, 'min_impurity_decrease': 0.0, 'min_impurity_split': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 100, 'n_iter_no_change': None, 'presort': 'deprecated', 'random_state': None, 'subsample': 1.0, 'tol': 0.0001, 'validation_fraction': 0.1, 'verbose': 0, 'warm_start': False}
[[30642  8577]
 [  180  1548]]
              precision    recall  f1-score   support

           0       0.99      0.78      0.87     39219
           1       0.15      0.90      0.26      1728

    accuracy                           0.79     40947
   macro avg       0.57      0.84      0.57     40947
weighted avg       0.96      0.79      0.85     40947

ROC AUC prob1: 0.8907457606290601
MCC: 0.3155510213978344
F2: 0.45430533544638135
PR_AUC: 0.2628961420578432
CPU times: user 5.71 s, sys: 188 ms, total

<h1 align='center'>Hyperparameter Tuning with Cross Validation</h1>

In [13]:
%%time
def create_model(trial):
    n_estimators = trial.suggest_int('n_estimators', 10, 200)
    subsample = trial.suggest_float('subsample', 0.8, 1.0)
    max_depth = trial.suggest_int('max_depth',3,20)
    max_features = trial.suggest_int('max_features',10,20)
    learning_rate = trial.suggest_uniform('learning_rate',0.08,0.2)
    min_samples_leaf = trial.suggest_int('min_samples_leaf',1,50)
    model = GradientBoostingClassifier(n_estimators=n_estimators, subsample=subsample, 
                           max_depth=max_depth, learning_rate=learning_rate, 
                           min_samples_leaf=min_samples_leaf)
#    model = GradientBoostingClassifier(n_estimators=n_estimators, subsample=subsample, 
#                           learning_rate=learning_rate, 
#                           min_samples_leaf=min_samples_leaf)
#    model = GradientBoostingClassifier(n_estimators=n_estimators, learning_rate=learning_rate) 
    return model

def objective(trial):
    model = create_model(trial)
    
    mcc_list = []
    roc_list = []
    pr_list = []
    f2_list = []
    cv = StratifiedKFold(n_splits=4, shuffle=True, random_state=11)
    for idx, (train_idx, test_idx) in enumerate(cv.split(X, y)):
        X_train_c, X_test_c = X.iloc[train_idx], X.iloc[test_idx]
        y_train_c, y_test_c = y[train_idx], y[test_idx]
        sampler = RandomUnderSampler(sampling_strategy=1.0, random_state=7)
        X_res, y_res = sampler.fit_resample(X_train_c, y_train_c)

#        weights = class_weight.compute_sample_weight('balanced', y=y_train)
#        model.fit(X_train, y_train, sample_weight=weights)
        model.fit(X_res, y_res)
        y_pred = model.predict(X_test_c)
        probs = model.predict_proba(X_test_c)
        prob1 = probs[:, 1]  # Only positives
        precision, recall, pr_thresh = precision_recall_curve(y_test_c, prob1)
        roc = roc_auc_score(y_test_c, prob1)
        roc_list.append(roc)
        mcc = matthews_corrcoef(y_test_c, y_pred)  # MWB
        mcc_list.append(mcc)
        f2 = fbeta_score(y_test_c, y_pred, beta=2.0, average=None)[1]
        pr_auc = auc(recall, precision)
        pr_list.append(pr_auc)
        avg_precision = average_precision_score(y_test_c, prob1)
#        print(f'ROC: {roc}')
#        print(f'PR_AUC: {pr_auc}')
#        print(f'avg_prec: {avg_precision}')
        
    print(f'MCC: {mcc_list}')
    print(f'ROC: {roc_list}')
    print(f'PR_AUC: {pr_list}')
    return np.mean(mcc_list)

sampler = TPESampler(seed=7)
study = optuna.create_study(sampler=sampler,direction='maximize')
#study.optimize(objective,n_trials=60)
study.optimize(objective,n_trials=25)
#study.optimize(objective,n_trials=5)

[I 2021-12-09 11:23:25,197] A new study created in memory with name: no-name-83ac6fac-bd87-4e13-a472-b95d747494ba
[I 2021-12-09 11:23:37,400] Trial 0 finished with value: 0.3152829671332036 and parameters: {'n_estimators': 24, 'subsample': 0.955983758448023, 'max_depth': 10, 'max_features': 17, 'learning_rate': 0.1973587414395923, 'min_samples_leaf': 27}. Best is trial 0 with value: 0.3152829671332036.


MCC: [0.3138936185267518, 0.31420967625081936, 0.32035485850998396, 0.31267371524525933]
ROC: [0.889038584779933, 0.88843264046128, 0.8933235738836873, 0.8892968412669272]
PR_AUC: [0.25017462769277027, 0.2507082785687574, 0.2508721096895517, 0.25410747744698314]


[I 2021-12-09 11:24:07,170] Trial 1 finished with value: 0.31563438937732824 and parameters: {'n_estimators': 105, 'subsample': 0.8144102266719524, 'max_depth': 7, 'max_features': 15, 'learning_rate': 0.16150759953451288, 'min_samples_leaf': 41}. Best is trial 1 with value: 0.31563438937732824.


MCC: [0.3146507782962976, 0.3149759738994252, 0.3201994417363983, 0.3127113635771919]
ROC: [0.8906296321794532, 0.8855576294120447, 0.8952697798312357, 0.8874103497105011]
PR_AUC: [0.26807612257059, 0.2376509234007379, 0.2467480186133173, 0.25513550793652606]


[I 2021-12-09 11:24:33,109] Trial 2 finished with value: 0.32035225259792793 and parameters: {'n_estimators': 82, 'subsample': 0.813187269381181, 'max_depth': 8, 'max_features': 20, 'learning_rate': 0.10560624242958987, 'min_samples_leaf': 23}. Best is trial 2 with value: 0.32035225259792793.


...

MCC: [0.318453810015736, 0.31820750097023204, 0.32128512539987825, 0.3145725576920922]
ROC: [0.8914366407094629, 0.8879781344130985, 0.8934805473961203, 0.892269633392151]
PR_AUC: [0.25323851815539183, 0.2494129906659161, 0.24715354026872283, 0.26404838420760207]


[I 2021-12-09 11:39:16,110] Trial 23 finished with value: 0.31856386681302135 and parameters: {'n_estimators': 58, 'subsample': 0.8265540110211977, 'max_depth': 18, 'max_features': 14, 'learning_rate': 0.08190386388453622, 'min_samples_leaf': 40}. Best is trial 2 with value: 0.32035225259792793.


MCC: [0.31710164019681464, 0.3175834766910986, 0.3217024938288767, 0.3178678565352953]
ROC: [0.8920012247652461, 0.8866777668985307, 0.8957414334428059, 0.8898455907101985]
PR_AUC: [0.2540962794681403, 0.2441543272593494, 0.2548644739892797, 0.251754640289544]


[I 2021-12-09 11:39:56,259] Trial 24 finished with value: 0.3165383440163901 and parameters: {'n_estimators': 79, 'subsample': 0.8799731267520511, 'max_depth': 17, 'max_features': 11, 'learning_rate': 0.11764679805898987, 'min_samples_leaf': 49}. Best is trial 2 with value: 0.32035225259792793.


MCC: [0.3171920886697995, 0.3135512030302957, 0.3200714342494366, 0.3153386501160285]
ROC: [0.8905542423964261, 0.8866425368364509, 0.8915925305468863, 0.8863563053966117]
PR_AUC: [0.2517535011202005, 0.24690433907681875, 0.24431934366495422, 0.24838861276610702]
CPU times: user 16min 9s, sys: 22.3 s, total: 16min 31s
Wall time: 16min 31s


In [14]:
best = study.best_trial
best

FrozenTrial(number=2, values=[0.32035225259792793], datetime_start=datetime.datetime(2021, 12, 9, 11, 24, 7, 171647), datetime_complete=datetime.datetime(2021, 12, 9, 11, 24, 33, 109668), params={'n_estimators': 82, 'subsample': 0.813187269381181, 'max_depth': 8, 'max_features': 20, 'learning_rate': 0.10560624242958987, 'min_samples_leaf': 23}, distributions={'n_estimators': IntUniformDistribution(high=200, low=10, step=1), 'subsample': UniformDistribution(high=1.0, low=0.8), 'max_depth': IntUniformDistribution(high=20, low=3, step=1), 'max_features': IntUniformDistribution(high=20, low=10, step=1), 'learning_rate': UniformDistribution(high=0.2, low=0.08), 'min_samples_leaf': IntUniformDistribution(high=50, low=1, step=1)}, user_attrs={}, system_attrs={}, intermediate_values={}, trial_id=2, state=TrialState.COMPLETE, value=None)

In [15]:
%%time
gb_params = study.best_params
#gb_params['random_state'] = 42
gb = GradientBoostingClassifier(**gb_params)
gb.fit(X_rus, y_rus)
y_pred = gb.predict(X_test)
probs = gb.predict_proba(X_test)
prob1 = probs[:, 1]  # Only positives
precision, recall, pr_thresh = precision_recall_curve(y_test, prob1)
print(confusion_matrix(y_pred=y_pred, y_true=y_test))
print(classification_report(y_test, y_pred))
print(f'ROC AUC prob1: {roc_auc_score(y_test, prob1)}')
print(f'MCC: {matthews_corrcoef(y_test, y_pred)}')
print(f'F2: {fbeta_score(y_test, y_pred, beta=2.0, average=None)[1]}')
print(f'PR_AUC: {auc(recall, precision)}')

[[30774  8445]
 [  184  1544]]
              precision    recall  f1-score   support

           0       0.99      0.78      0.88     39219
           1       0.15      0.89      0.26      1728

    accuracy                           0.79     40947
   macro avg       0.57      0.84      0.57     40947
weighted avg       0.96      0.79      0.85     40947

ROC AUC prob1: 0.8929236056810146
MCC: 0.3174852482145634
F2: 0.45677770546121543
PR_AUC: 0.26745242409145414
CPU times: user 1.69 s, sys: 82.4 ms, total: 1.78 s
Wall time: 1.77 s


In [16]:
%%time
gb_params = study.best_params
gb_params['random_state'] = 42
gb_weight = GradientBoostingClassifier(**gb_params)
weights = class_weight.compute_sample_weight('balanced', y=y_train)
gb_weight.fit(X_train, y_train, sample_weight=weights)
y_pred = gb_weight.predict(X_test)
probs = gb_weight.predict_proba(X_test)
prob1 = probs[:, 1]  # Only positives
precision, recall, pr_thresh = precision_recall_curve(y_test, prob1)
print(confusion_matrix(y_pred=y_pred, y_true=y_test))
print(classification_report(y_test, y_pred))
print(f'ROC AUC prob1: {roc_auc_score(y_test, prob1)}')
print(f'MCC: {matthews_corrcoef(y_test, y_pred)}')
print(f'F2: {fbeta_score(y_test, y_pred, beta=2.0, average=None)[1]}')
print(f'PR_AUC: {auc(recall, precision)}')

[[31741  7478]
 [  258  1470]]
              precision    recall  f1-score   support

           0       0.99      0.81      0.89     39219
           1       0.16      0.85      0.28      1728

    accuracy                           0.81     40947
   macro avg       0.58      0.83      0.58     40947
weighted avg       0.96      0.81      0.87     40947

ROC AUC prob1: 0.8902099886274888
MCC: 0.3211049909932943
F2: 0.4634300126103405
PR_AUC: 0.27181562205926146
CPU times: user 12.9 s, sys: 146 ms, total: 13 s
Wall time: 13 s


In [24]:
# No Sitenum - 5
coeffs_wgt = pd.Series(data=gb_weight.feature_importances_,
                   index=X_test.columns.values).sort_values(ascending=False)
coeffs_wgt[0:20]

Delmode           0.231738
CS_FTP            0.080058
Admefface         0.072331
TrialLabor        0.053035
Lac_Min           0.034673
Delfetalpos       0.031561
transfus_yes      0.031359
CS_NRFHT          0.026901
prelaborCD        0.024939
Insurance         0.020519
BESTGA            0.020229
Hxanemia          0.019345
new_BMI           0.015959
HospElectInd      0.014446
new_age           0.013785
Admpresent        0.011479
AdmSBP            0.011131
Admcontract       0.011049
AdmDBP            0.010503
Inoxy_incrdose    0.010442
dtype: float64

In [1]:
plt.rcParams["figure.figsize"] = (14, 8)
x_labels = coeffs_wgt.index.values[0:25]
print(x_labels)
fig, ax = plt.subplots(1,1)  # Create a figure and an axes.
ax.set_title('Top Predictors: All Sites combined; target: High_EBLoss, vars: PI')
ax.bar(x_labels, coeffs_wgt.values[0:25])
ax.set_ylabel('Coeff')
ax.set_xlabel('Variable')
plt.draw()
ax.set_xticklabels(ax.get_xticklabels(), rotation=90, fontsize=16)
#plt.xlim(1,20)
plt.show()

NameError: name 'plt' is not defined

In [26]:
# No Sitenum - 5
coeffs = pd.Series(data=gb.feature_importances_,
                   index=X_test.columns.values).sort_values(ascending=False)
coeffs[0:20]

Delmode           0.261331
Lac_Min           0.096959
TrialLabor        0.042401
Delfetalpos       0.039474
Analgesia         0.037037
CS_FTP            0.036993
transfus_yes      0.029821
CS_NRFHT          0.025537
Insurance         0.021843
Dilat_lst         0.020394
Hxanemia          0.018626
new_BMI           0.018430
BESTGA            0.017936
HospElectInd      0.015323
Admcontract       0.014507
Inoxy_incrdose    0.014255
prelaborCD        0.014023
Augment           0.013633
AdmSBP            0.013415
new_age           0.012889
dtype: float64

In [4]:
plt.rcParams["figure.figsize"] = (6, 4)
sns.heatmap(confusion_matrix(y_test, y_pred), cmap='magma', annot=True, fmt='g') # 25
plt.show()

NameError: name 'plt' is not defined

In [20]:
gb_params # 5 runs

{'n_estimators': 82,
 'subsample': 0.813187269381181,
 'max_depth': 8,
 'max_features': 20,
 'learning_rate': 0.10560624242958987,
 'min_samples_leaf': 23,
 'random_state': 42}

In [3]:
from optuna.visualization import plot_optimization_history

plotly_config = {"staticPlot": True}

fig = plot_optimization_history(study)
fig.show(config=plotly_config)

NameError: name 'study' is not defined

In [2]:
from optuna.visualization import plot_param_importances

fig = plot_param_importances(study)
fig.show(config=plotly_config)

NameError: name 'study' is not defined

In [23]:
np.mean([0.3149689279176914, 0.31449457458602254, 0.31066798538596907, 0.3219519143488398])

0.3155208505596307

<h1 align='center'>Thanks and make sure to learn!</h1>